# **🦙 Llama-2-7B model Test on Math Dataset**

In [ ]:
!pip install -q transformers accelerate langchain
!pip install langchain-community langchain-core
!pip install datasets
!pip install requests
!huggingface-cli login

In [2]:
from langchain.llms import HuggingFacePipeline
from langchain.prompts import PromptTemplate
from transformers import AutoTokenizer, AutoModelForCausalLM
from langchain.chains import ConversationChain
from datasets import load_dataset
import pandas as pd
import re
import transformers
import torch
import warnings
warnings.filterwarnings('ignore')
import accelerate

In [3]:
model_name = "meta-llama/Llama-2-7b-chat-hf"

In [4]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [5]:
# Load the model and move it to the GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(device)

cuda


In [6]:
pipeline = transformers.pipeline(
    "text-generation",
    model = model_name,
    tokenizer = tokenizer,
    torch_dtype = torch.bfloat16,
    trust_remote_code=True,
    device_map = "auto",
    max_new_tokens = 500,  # Adjust this
    # max_length=1000,
    do_sample = True,
    top_k = 10,
    num_return_sequences = 1,
    eos_token_id = tokenizer.eos_token_id
    )

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [7]:
llm=HuggingFacePipeline(pipeline = pipeline, model_kwargs={'temperature':0.7})

In [8]:
prompt_template = """Extract and return only the final answer from the following text:

{input}

Final Answer:"""

prompt = PromptTemplate(template=prompt_template, input_variables=['input'])

In [9]:
from langchain.chains import LLMChain

chain = LLMChain(
    llm=llm,
    prompt=prompt
)

In [10]:
input = "the capital of Egypt is Cairo which is located at the middle of Egypt"

In [11]:
response = chain.run(input)

# Assumption: The final answer will be right after "Final Answer:"
if "Final Answer:" in response:
    final_answer = response.split("Final Answer:")[1].strip()
else:
    final_answer = response.strip()

print(final_answer.strip())

Cairo


# **📊 Load the Dataset**

In [12]:
# Load MATH dataset
dataset = load_dataset("lighteval/MATH")

print(dataset)

DatasetDict({
    train: Dataset({
        features: ['problem', 'level', 'type', 'solution'],
        num_rows: 7500
    })
    test: Dataset({
        features: ['problem', 'level', 'type', 'solution'],
        num_rows: 5000
    })
})


In [13]:
# Access first data from the training set

n = 0

sample_problem = dataset['train'][n]['problem']
print(f"Problem {n}: {sample_problem}\n")

sample_solution = dataset['train'][n]['solution']
print(f"Solution {n}: {sample_solution}")

Problem 0: Let \[f(x) = \left\{
\begin{array}{cl} ax+3, &\text{ if }x>2, \\
x-5 &\text{ if } -2 \le x \le 2, \\
2x-b &\text{ if } x <-2.
\end{array}
\right.\]Find $a+b$ if the piecewise function is continuous (which means that its graph can be drawn without lifting your pencil from the paper).

Solution 0: For the piecewise function to be continuous, the cases must "meet" at $2$ and $-2$. For example, $ax+3$ and $x-5$ must be equal when $x=2$. This implies $a(2)+3=2-5$, which we solve to get $2a=-6 \Rightarrow a=-3$. Similarly, $x-5$ and $2x-b$ must be equal when $x=-2$. Substituting, we get $-2-5=2(-2)-b$, which implies $b=3$. So $a+b=-3+3=\boxed{0}$.


In [14]:
print(f"shape : {dataset.shape}\n")

print(f"train : {dataset['train'].shape[0]}")
print(f"test  : {dataset['test'].shape[0]}")

shape : {'train': (7500, 4), 'test': (5000, 4)}

train : 7500
test  : 5000


# **🔎 Data Preprocessing**

In [20]:
train_sets = dataset['train'].shape[0]
temp_n = 10

# List of prompts
prompts = []

# temp_n here will be placed with train_sets to train on actual dataset
for i in range(train_sets):
  sample_i = dataset['train'][i]['solution']
  prompts.append(sample_i)

In [21]:
print(len(prompts))

7500


In [19]:
# Empty list to hold responses
responses = []

# Change temp_n to train_sets to use the entire dataset
for i in range(temp_n):

    problem = dataset['train'][i]['problem']
    level = dataset['train'][i]['level']
    solution = dataset['train'][i]['solution']

    response = chain.run(solution)

    if "Final Answer:" in response:
        final_answer = response.split("Final Answer:")[1].strip()
    else:
        final_answer = response.strip()

    cleaned_response = final_answer.strip()
    responses.append(cleaned_response)

    data = {
        "problem": [problem],
        "level": [level],
        "solution": [solution],
        "final_solution": [cleaned_response]
    }

    df = pd.DataFrame(data)

    df.to_csv("cleaned_data.csv", mode='a', header=not pd.io.common.file_exists('cleaned_data.csv'), index=False)

print("Data has been saved to cleaned_data.csv")

Data has been saved to cleaned_data.csv


In [18]:
# Print all responses
for i, response_text in enumerate(responses):
    print(f"Prompt {i + 1}: {prompts[i]}\n")
    print(f"Response {i + 1}: {response_text}\n")
    print()

Prompt 1: For the piecewise function to be continuous, the cases must "meet" at $2$ and $-2$. For example, $ax+3$ and $x-5$ must be equal when $x=2$. This implies $a(2)+3=2-5$, which we solve to get $2a=-6 \Rightarrow a=-3$. Similarly, $x-5$ and $2x-b$ must be equal when $x=-2$. Substituting, we get $-2-5=2(-2)-b$, which implies $b=3$. So $a+b=-3+3=\boxed{0}$.

Response 1: $a=-3$ and $b=3$.


Prompt 2: Let $x$ be the number of band members in each row for the original formation, when two are left over.  Then we can write two equations from the given information: $$rx+2=m$$ $$(r-2)(x+1)=m$$ Setting these equal, we find: $$rx+2=(r-2)(x+1)=rx-2x+r-2$$ $$2=-2x+r-2$$ $$4=r-2x$$ We know that the band has less than 100 members.  Based on the first equation, we must have $rx$ less than 98.  We can guess and check some values of $r$ and $x$ in the last equation.  If $r=18$, then $x=7$, and $rx=126$ which is too big.  If $r=16$, then $x=6$, and $rx=96$, which is less than 98.  Checking back in t